# Email Generator

In [23]:
pip install -qU langchain_community beautifulsoup4 chromaDB

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain_groq import ChatGroq

In [25]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key = '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'
    # other params...
)

res = llm.invoke("Hello, World!")
print(res.content)

Hello. How can I assist you today?


In [26]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/87135988375003846-generative-ai-evaluation-clinical-scientist")
page_data = loader.load().pop().page_content
print(page_data)

Generative AI Evaluation Clinical Scientist — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results2,556  jobs matchedChip Package Signal and Power Integrity EngineerSunnyvale, CA, USASoftware Engineer, Content Safety On Device, Hades PlatformBelo Horizonte, State of Minas Gerais, Brazil; São Paulo, State of São Paulo, BrazilSoftware Engineering Manager II, Site Reliability Engineering, ShoppingDublin, IrelandAccount Strategist, Engage, Google Customer Solutions, Vietnam MarketHo Chi Minh, Ho Chi Minh City, VietnamSoC Power and Performance Lead, TPU, Google CloudBengaluru, Karnataka, IndiaSenior UX Researcher, gTech Users and ProductsBoulder, CO, USA; Atlanta, GA, USACloud Consultant, Application Modernization E

In [27]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JOSN format containing
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLES):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})

print(res.content)

```json
{
  "role": "Generative AI Evaluation Clinical Scientist",
  "experience": "3 years of experience applying qualitative and quantitative research methods relevant to health AI model validation",
  "skills": [
    "Qualitative and quantitative research methods",
    "Clinical study design and execution",
    "Programming skills (e.g., Python/Colab/Jupyter notebooks, R)",
    "AI/ML applied to clinical or health problems",
    "Clinical evaluation of generative AI models",
    "Clinical research, digital health, real-world evaluations and working with large and complex healthcare datasets"
  ],
  "description": "Develop, validate, and deploy standardized rating instruments to assess the quality, accuracy, robustness, and efficacy of AI models, ensuring alignment with Google’s AI Principles, regulatory guidelines, and industry best practices. Design and execute evaluations of AI models in health-related areas, with a focus on assessing information quality, clinical impact, safety, 

In [28]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Generative AI Evaluation Clinical Scientist',
 'experience': '3 years of experience applying qualitative and quantitative research methods relevant to health AI model validation',
 'skills': ['Qualitative and quantitative research methods',
  'Clinical study design and execution',
  'Programming skills (e.g., Python/Colab/Jupyter notebooks, R)',
  'AI/ML applied to clinical or health problems',
  'Clinical evaluation of generative AI models',
  'Clinical research, digital health, real-world evaluations and working with large and complex healthcare datasets'],
 'description': 'Develop, validate, and deploy standardized rating instruments to assess the quality, accuracy, robustness, and efficacy of AI models, ensuring alignment with Google’s AI Principles, regulatory guidelines, and industry best practices. Design and execute evaluations of AI models in health-related areas, with a focus on assessing information quality, clinical impact, safety, and potential biases.'}

In [29]:
import pandas as pd

df = pd.read_csv("techstack_links.csv")
df

,techstack,links
0,HTML,https://portfolio.example.com/html
1,CSS,https://portfolio.example.com/css
2,JavaScript,https://portfolio.example.com/javascript
3,Node.js,https://portfolio.example.com/nodejs
4,React,https://portfolio.example.com/react
5,Angular,https://portfolio.example.com/angular
6,Vue.js,https://portfolio.example.com/vuejs
7,Express.js,https://portfolio.example.com/expressjs
8,MongoDB,https://portfolio.example.com/mongodb
9,MySQL,https://portfolio.example.com/mysql


In [30]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstor')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["techstack"],
                      metadatas={'links': row["links"]},
                      ids=[str(uuid.uuid4())])

In [31]:
job = josn_res
job['skills']

NameError: name 'josn_res' is not defined

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohandas, a Business Development Executive at Codevolve. Codevolve is an AI & Software company that specializes
    in the seamless integration of business processes through automated tools. Leveraging our extensive experience, we have 
    empowered businesses by optimizing their workflows and enhancing operational efficiency.

    Your job is to write a cold email to the client regarding the job mentioned above describing the capabilities in fulfilling
    their needs.
    Also add the most relavant once from the following links to showcase Codevolve's portfolio: {link_list}
    Remember you are Mohandas, BDE at Codevolve.

    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)